In [160]:
from collections import defaultdict
from time import sleep
# Word Search
from typing import List

from linked_list.reverse_linked_list import result
from linked_list.utils import ListNode, print_linked_list, create_linked_list
from stack.minimum_stack import MinStack


def word_search(board: List[List[str]], word: str) -> bool:
    rows, cols = len(board), len(board[0])

    word_size = len(word)

    def backtrack(i, j, index):
        cell = board[i][j]
        if cell != word[index]:
            return False
        if word_size - 1 == index:
            return True

        board[i][j] = "Do not visit again"
        for i_off, j_off in [(0, -1), (-1, 0), (1, 0), (0, 1)]:
            updated_i, updated_j = i + i_off, j + j_off
            if backtrack(updated_i, updated_j, index + 1):
                return True

        board[i][j] = cell

        return False

    for i in range(rows):
        for j in range(cols):
            if backtrack(i, j, 0):
                return True

    return False


print(word_search(board=[["A", "B", "C", "E"], ["S", "F", "C", "S"], ["A", "D", "E", "E"]], word="ABCCED"))

None


In [45]:
# Two Sums

# Brute Force

def two_sums_bf(nums, target):
    size = len(nums)
    for fi in range(0, size):
        for si in range(fi + 1, size):
            first_word = nums[fi]
            second_word = nums[si]
            if first_word + second_word == target:
                return [fi, si]
    return None


def two_sums_opt(nums, target):
    storage = {}
    result = []
    for index, num in enumerate(nums):
        if num in storage:
            result.append(storage.get(num))
            result.append(index)
        else:
            diff = target - num
            storage[diff] = index
    return result


print(two_sums_opt(nums=[5, 2, 3, 4, 4, 19, 10, 8, 9], target=29))


[5, 6]


In [17]:
# Number of Islands

def number_of_islands(grid):
    rows, cols = len(grid), len(grid[0])

    def dfs(i, j):
        stk = [(i, j)]

        while stk:
            x, y = stk.pop()
            if x < rows and x >= 0 and y < cols and y >= 0 and grid[x][y] == "1":
                grid[x][y] = "0"
                for i_delta, j_delta in [(0, 1), (1, 0), (-1, 0), (0, -1)]:
                    r, c = x + i_delta, y + j_delta
                    stk.append((r, c))

    count = 0
    for i in range(rows):
        for j in range(cols):
            if grid[i][j] == "1":
                count += 1
                dfs(i, j)

    return count


print(number_of_islands(grid=[
    ["1", "1", "1", "1", "0"],
    ["1", "1", "0", "1", "0"],
    ["1", "1", "0", "0", "0"],
    ["0", "0", "0", "0", "0"]
]))

1


In [44]:
# Valid Parenthesis

def is_valid_parenthesis(s):
    close_open = {')': '(', '}': '{', ']': '['}
    stk = []

    for index, item in enumerate(s):
        if item in close_open and stk:
            if close_open.get(item) == stk[-1]:
                stk.pop()
            else:
                return False
        else:
            stk.append(item)
    if len(stk) == 0:
        return True
    else:
        return False


print(is_valid_parenthesis(s="({[]})"))

True


In [53]:
from collections import OrderedDict


class LRUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = OrderedDict()

    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1
        self.cache.move_to_end(key, last=True)
        return self.cache.get(key)

    def put(self, key: int, value: int) -> None:
        if len(self.cache) >= self.capacity:
            self.cache.popitem(last=False)
        self.cache[key] = value


lRUCache = LRUCache(2)
lRUCache.put(1, 1)  # cache is {1=1}
lRUCache.put(2, 2)  #  cache is {1=1, 2=2}
print(lRUCache.get(1))  #  return 1
lRUCache.put(3, 3)  #  LRU key was 2, evicts key 2, cache is {1=1, 3=3}
print(lRUCache.get(2))  #  returns -1 (not found)
lRUCache.put(4, 4)  #  LRU key was 1, evicts key 1, cache is {4=4, 3=3}
print(lRUCache.get(1))  #  return -1 (not found)
print(lRUCache.get(3))  #  return 3
print(lRUCache.get(4))  #  return 4

1
-1
-1
3
4


In [57]:
from collections import defaultdict


class LFUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = OrderedDict()
        self.freq_map = defaultdict(list)
        self.min_freq = 0

    def get(self, key):
        if key not in self.cache:
            return -1

        value, _ = self.cache[key]
        self._updated_frequency(key=key)
        return value

    def put(self, key, value):
        if key in self.cache:
            _, freq = self.cache[key]
            self.cache[key] = [value, freq]
            self._updated_frequency(key=key)
        else:
            if len(self.cache) >= self.capacity:
                lfu_key = self.freq_map[self.min_freq].pop(0)
                if not self.freq_map[self.min_freq]:  # If no keys remain at this frequency
                    del self.freq_map[self.min_freq]
                del self.cache[lfu_key]
            self.cache[key] = [value, 1]
            self.freq_map[1].append(key)
            self.min_freq = 1

    def _updated_frequency(self, key):
        value, freq = self.cache[key]

        # Remove
        self.freq_map[freq].remove(key)
        if not self.freq_map[freq]:
            del self.freq_map[freq]
            if self.min_freq == freq:
                self.min_freq += 1

        # Increment
        new_freq = freq + 1
        self.freq_map[new_freq].append(key)
        self.cache[key] = [value, new_freq]


# Example usage
lfuCache = LFUCache(2)
lfuCache.put(1, 1)  # Cache is {1: 1}, frequency is {1: [1]}
lfuCache.put(2, 2)  # Cache is {1: 1, 2: 2}, frequency is {1: [1, 2]}
print(lfuCache.get(1))  # Return 1, Cache is {1: 1, 2: 2}, frequency is {1: [2], 2: [1]}
lfuCache.put(3, 3)  # Evicts key 2 (LFU), Cache is {1: 1, 3: 3}, frequency is {1: [3], 2: [1]}
print(lfuCache.get(2))  # Return -1 (not found)
print(lfuCache.get(3))  # Return 3, Cache is {1: 1, 3: 3}, frequency is {2: [1, 3]}
lfuCache.put(4, 4)  # Evicts key 1 (LFU), Cache is {4: 4, 3: 3}, frequency is {1: [4], 2: [3]}
print(lfuCache.get(1))  # Return -1 (not found)
print(lfuCache.get(3))  # Return 3, Cache is {4: 4, 3: 3}, frequency is {1: [4], 3: [3]}
print(lfuCache.get(4))  # Return 4, Cache is {4: 4, 3: 3}, frequency is {2: [4], 3: [3]}


1
-1
3
-1
3
4


In [85]:
# Merge two sorted lists
from typing import Optional


def mergeTwoLists(list1: Optional[ListNode], list2: Optional[ListNode]) -> Optional[ListNode]:
    dummy = ListNode()
    curr = dummy

    while list1 and list2:
        if list2.value > list1.value:
            curr.next = list1
            list1 = list1.next
        else:
            curr.next = list2
            list2 = list2.next
        curr = curr.next

    if list2:
        curr.next = list2
    elif list1:
        curr.next = list1
    return dummy.next


l1 = create_linked_list([1, 3, 5, 7, 50, 100])
l2 = create_linked_list([2, 4, 6, 8])

print_linked_list(l1)
print_linked_list(l2)

merged = mergeTwoLists(l1, l2)
print_linked_list(merged)

1 -> 3 -> 5 -> 7 -> 50 -> 100
2 -> 4 -> 6 -> 8
1 -> 2 -> 3 -> 4 -> 5 -> 6 -> 7 -> 8 -> 50 -> 100


In [ ]:
# Reverse a linked list

def reverseList(head: Optional[ListNode]) -> Optional[ListNode]:
    curr = head
    prev = None
    while curr:
        next = curr.next
        curr.next = prev
        prev = curr
        curr = next

    return prev


In [103]:
# All path from source to target

def allPathsSourceTarget(graph: List[List[int]]) -> List[List[int]]:
    graph_representation = {}
    for index, value in enumerate(graph):
        graph_representation[index] = value
    target = len(graph) - 1
    stk = []
    stk.append((0, [0]))
    result = []
    while stk:
        node, path = stk.pop()
        if path[-1] == target:
            result.append(path)
            continue
        if node in graph_representation:
            nei = graph_representation.get(node)
            for i in nei:
                updated_path = path[::]
                updated_path.append(i)
                stk.append((i, updated_path))
    return result


# print(allPathsSourceTarget(graph=[[1, 2], [3], [3], []]))
print(allPathsSourceTarget(graph=[[4, 3, 1], [3, 2, 4], [3], [4], []]))



{0: [4, 3, 1], 1: [3, 2, 4], 2: [3], 3: [4], 4: []}
[[0, 1, 4], [0, 1, 2, 3, 4], [0, 1, 3, 4], [0, 3, 4], [0, 4]]


In [105]:
# Add Digit
def addDigits(num: int) -> int:
    if num == 0:
        return 0
    elif num % 9 == 0:
        return 9
    else:
        return num % 9

In [106]:
# 4 SUM

def four_sum(nums, target):
    nums.sort()
    size = len(nums)
    result = []
    for index1 in range(size):
        if index1 > 0 and nums[index1] == nums[index1 - 1]:
            continue
        for index2 in range(index1 + 1, size):
            if index2 > index1 + 1 and nums[index2] == nums[index2 - 1]:
                continue

            remainder = nums[index1] + nums[index2]

            left = index2 + 1
            right = size - 1

            while right > left:
                total = remainder + nums[left] + nums[right]
                if total == target:
                    result.append([nums[index1], nums[index2], nums[left], nums[right]])

                    left, right = left + 1, right - 1

                    while right > left and nums[left] == nums[left - 1]:
                        left += 1

                    while right > left and nums[right] == nums[right + 1]:
                        right -= 1

                elif total > target:
                    right -= 1
                else:
                    left += 1
    return result

In [107]:
# MinStack

class MinStack(object):

    def __init__(self):
        self.stack = []
        self.min_stack = []

    def push(self, val):
        """
        :type val: int
        :rtype: None
        """
        self.stack.append(val)
        if self.min_stack:
            if self.min_stack[-1] >= val:
                self.min_stack.append(val)
            else:
                self.min_stack.append(self.min_stack[-1])
        else:
            self.min_stack.append(val)

    def pop(self):
        """
        :rtype: None
        """
        self.stack.pop()
        self.min_stack.pop()

    def top(self):
        """
        :rtype: int
        """
        return self.stack[-1]

    def getMin(self):
        """
        :rtype: int
        """
        return self.min_stack[-1]

In [ ]:
# Two City Scheduling

def twoCitySchedCost(costs: List[List[int]]) -> int:
    costs.sort(key=lambda x: x[0] - x[1])
    size = len(costs)
    mid = size // 2

    total_cost = 0
    for i in range(mid):
        total_cost += i[0][0]

    for j in range(mid, mid * 2):
        total_cost += j[0][1]

    return total_cost




In [117]:
# Course Schedule II

# Python
from collections import defaultdict
from typing import List


def findOrder(numCourses: int, prerequisites: List[List[int]]) -> List[int]:
    graph = defaultdict(list)
    # Build the graph: edge from prerequisite to course.
    for course, prereq in prerequisites:
        graph[prereq].append(course)

    # State: 0 = unvisited, 1 = visiting, 2 = visited.
    state = [0] * numCourses
    order = []
    UNVISITED, VISITING, VISITED = 0, 1, 2

    def dfs(node: int) -> bool:
        if state[node] == VISITING:
            return False
        if state[node] == VISITED:
            return True
        state[node] = VISITING
        for neighbor in graph[node]:
            if not dfs(neighbor):
                return False
        state[node] = VISITED
        order.append(node)
        return True

    # Visit every course.
    for course in range(numCourses):
        if state[course] == UNVISITED:
            if not dfs(course):
                return []
    # Reverse order to get a valid topological order.
    return order[::-1]


print(findOrder(numCourses=4, prerequisites=[[1, 0], [2, 0], [3, 1], [3, 2]]))

[0, 1, 2, 3]


In [128]:
# Longest Consecutive Sequence

def longestConsecutive(nums):
    unique_nums = set(nums)

    result = 0

    for num in unique_nums:
        if num - 1 not in unique_nums:
            l = 0
            next_num = num
            while next_num in unique_nums:
                l += 1
                next_num += 1
            result = max(result, l)
    return result


nums = [2, 20, 21, 22, 4, 10, 3, 4, 5, 23, 24, 25, 26, 27, 28, ]
print(longestConsecutive(nums=nums))

9


In [143]:
# Add Two Numbers

def addTwoNumbers(l1: Optional[ListNode], l2: Optional[ListNode]) -> Optional[ListNode]:
    dummy = ListNode()
    curr = dummy
    carry = 0
    r1 = l1
    r2 = l2
    while r2 or r1:
        summed_up = 0
        if r2 and r1:
            addition = r1.value + r2.value + carry
            carry = addition // 10
            summed_up = addition % 10
        elif r2 and not r1:
            addition = r2.value + carry
            carry = addition // 10
            summed_up = addition % 10
        elif r1 and not r2:
            addition = r1.value + carry
            carry = addition // 10
            summed_up = addition % 10
        r1 = r1.next if r1 else None
        r2 = r2.next if r2 else None
        curr.next = ListNode(value=summed_up)
        curr = curr.next

    if carry != 0:
        curr.next = ListNode(value=carry)
    return dummy.next


def reverse(l: Optional[ListNode]):
    prev, curr = None, l

    while curr:
        next = curr.next
        curr.next = prev
        prev = curr
        curr = next

    return prev


l1 = create_linked_list([2, 4, 3, 3])
l2 = create_linked_list([5, 6, 4])

print("List 1:")
print_linked_list(l1)
print("List 2:")
print_linked_list(l2)

result = addTwoNumbers(l1, l2)
print("Resultant List:")
print_linked_list(result)

List 1:
2 -> 4 -> 3 -> 3
List 2:
5 -> 6 -> 4
7 -> 9 -> 9 -> 2
Resultant List:



In [155]:
# Minimum Remove to Make Valid Parentheses

def minRemovalToMakeValidParentheses(s):
    stk = []
    bad_indices = set()
    for index, char in enumerate(s):
        if char == "(":
            stk.append((index, char))
        elif char == ")":
            if len(stk) > 0 and stk[-1][1] == "(":
                stk.pop()
            else:
                bad_indices.add(index)

    for item, _ in stk:
        bad_indices.add(item)

    result = []
    for index, item in enumerate(s):
        if index not in bad_indices:
            result.append(item)

    return "".join(result)


print(minRemovalToMakeValidParentheses(s="lee(t(c)o)de)"))

lee(t(c)o)de


In [ ]:
# Average of subtree
from binary_tree.util import TreeNode


def averageOfSubtree(root: TreeNode) -> int:
    dfs(root, 0)



In [158]:
import heapq
from collections import defaultdict


class Leaderboard:
    def __init__(self):
        self.scores = defaultdict(int)  # Player ID to score mapping
        self.sorted_scores = []  # Min-heap to maintain top scores

    def add_score(self, player_id, score):
        # Update the player's score
        self.scores[player_id] += score

    def top_k(self, k):
        # Use a heap to get the top k players
        print(self.scores.items())
        return heapq.nlargest(k, self.scores.items(), key=lambda x: x[1])

    def reset(self, player_id):
        # Reset the player's score to zero
        if player_id in self.scores:
            self.scores[player_id] = 0


# Example Usage
lb = Leaderboard()
lb.add_score(1, 50)
lb.add_score(2, 70)
lb.add_score(3, 30)
lb.add_score(1, 20)

print(lb.top_k(2))  # Top 2 players: [(2, 70), (1, 70)]
lb.reset(1)
print(lb.top_k(2))  # Top 2 players: [(2, 70), (3, 30)]


dict_items([(1, 70), (2, 70), (3, 30)])
[(1, 70), (2, 70)]
dict_items([(1, 0), (2, 70), (3, 30)])
[(2, 70), (3, 30)]


In [121]:
# Heaters

import bisect


def findRadius(houses, heaters):
    houses.sort()
    heaters.sort()

    radius = 0

    for house in houses:
        index = bisect.bisect_left(heaters, house)

        if index == 0:
            min_dist = heaters[0] - house
        elif index == len(heaters):
            min_dist = house - heaters[-1]
        else:
            min_dist = min(house - heaters[index - 1], heaters[index] - house)

        radius = max(min_dist, radius)

    return radius


# Test example
houses = [1, 5, 10]
heaters = [4, 8]
print(findRadius(houses, heaters))  # Output: 3


3
